In [1]:
"""
notebook by - Will Sanger
ported from - Kartikeya Bhardwaj (twitter: @kartike_ya)
ported from - https://gist.github.com/nateraw/c989468b74c616ebbc6474aa8cdd9e53
"""

'\nnotebook by - Will Sanger\nported from - Kartikeya Bhardwaj (twitter: @kartike_ya)\nported from - https://gist.github.com/nateraw/c989468b74c616ebbc6474aa8cdd9e53\n'

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
import torch
from transformers import CLIPTextModel, CLIPTokenizer
from transformers import logging
from diffusers import AutoencoderKL, UNet2DConditionModel, LMSDiscreteScheduler
from tqdm.auto import tqdm
from torch import autocast
from PIL import Image
from matplotlib import pyplot as plt
import numpy
from torchvision import transforms as tfms
import numpy as np
import os
import gc
from pathlib import Path
import pandas as pd
from einops import rearrange

# For video display:
from IPython.display import HTML
from base64 import b64encode

# Supress some unnecessary warnings when loading the CLIPTextModel
logging.set_verbosity_error()

# Set device
torch_device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
# Load the autoencoder model which will be used to decode the latents into image space. 
vae = AutoencoderKL.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="vae")

# Load the tokenizer and text encoder to tokenize and encode the text. 
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")
text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14")

# The UNet model for generating the latents.
unet = UNet2DConditionModel.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="unet")

# The noise scheduler
scheduler = LMSDiscreteScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", num_train_timesteps=1000)

# To the GPU we go!
vae = vae.to(torch_device)
text_encoder = text_encoder.to(torch_device)
unet = unet.to(torch_device);

In [5]:
def diffuse(text_embeddings, init_noise):

    height = 512                       # default height of Stable Diffusion
    width = 512                         # default width of Stable Diffusion
    num_inference_steps = 20            # Number of denoising steps
    guidance_scale = 7.5                # Scale for classifier-free guidance
    generator = torch.manual_seed(42)   # Seed generator to create the inital latent noise
    batch_size = 1

    # Prep Scheduler
    scheduler.set_timesteps(num_inference_steps)

    # Prep latents
    latents = init_noise
    latents = latents.to(torch_device)
    latents = latents * scheduler.init_noise_sigma # Scaling (previous versions did latents = latents * self.scheduler.sigmas[0]

    # Loop
    with autocast("cuda"):
        for i, t in tqdm(enumerate(scheduler.timesteps)):
            # expand the latents if we are doing classifier-free guidance to avoid doing two forward passes.
            latent_model_input = torch.cat([latents] * 2)
            sigma = scheduler.sigmas[i]
            # Scale the latents (preconditioning):
            # latent_model_input = latent_model_input / ((sigma**2 + 1) ** 0.5) # Diffusers 0.3 and below
            latent_model_input = scheduler.scale_model_input(latent_model_input, t)

            # predict the noise residual
            with torch.no_grad():
                noise_pred = unet(latent_model_input, t, encoder_hidden_states=text_embeddings).sample

            # perform guidance
            noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
            noise_pred = noise_pred_uncond + guidance_scale * (noise_pred_text - noise_pred_uncond)

            # compute the previous noisy sample x_t -> x_t-1
            # latents = scheduler.step(noise_pred, i, latents)["prev_sample"] # Diffusers 0.3 and below
            latents = scheduler.step(noise_pred, t, latents).prev_sample

    # scale and decode the image latents with vae
    latents = 1 / 0.18215 * latents
    with torch.no_grad():
        image = vae.decode(latents).sample

    # Display
    image = (image / 2 + 0.5).clamp(0, 1)
    image = image.detach().cpu().permute(0, 2, 3, 1).numpy()
    images = (image * 255).round().astype("uint8")
    pil_images = [Image.fromarray(image) for image in images]
    return pil_images[0]

In [6]:
def slerp(t, v0, v1, DOT_THRESHOLD=0.9995):
    """ helper function to spherically interpolate two arrays v1 v2 """

    if not isinstance(v0, np.ndarray):
        inputs_are_torch = True
        input_device = v0.device
        v0 = v0.cpu().numpy()
        v1 = v1.cpu().numpy()

    dot = np.sum(v0 * v1 / (np.linalg.norm(v0) * np.linalg.norm(v1)))
    if np.abs(dot) > DOT_THRESHOLD:
        v2 = (1 - t) * v0 + t * v1
    else:
        theta_0 = np.arccos(dot)
        sin_theta_0 = np.sin(theta_0)
        theta_t = theta_0 * t
        sin_theta_t = np.sin(theta_t)
        s0 = np.sin(theta_0 - theta_t) / sin_theta_0
        s1 = sin_theta_t / sin_theta_0
        v2 = s0 * v0 + s1 * v1

    if inputs_are_torch:
        v2 = torch.from_numpy(v2).to(input_device)

    return v2

In [7]:
def embed(prompt):
    text_input = tokenizer(prompt, padding="max_length", max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt")
    with torch.no_grad():
        text_embeddings = text_encoder(text_input.input_ids.to(torch_device))[0]
    max_length = text_input.input_ids.shape[-1]
    uncond_input = tokenizer(
      [""] * batch_size, padding="max_length", max_length=max_length, return_tensors="pt"
    )
    with torch.no_grad():
        uncond_embeddings = text_encoder(uncond_input.input_ids.to(torch_device))[0] 
    text_embeddings = torch.cat([uncond_embeddings, text_embeddings])
    return text_embeddings

In [8]:
ins_path = Path("../clean_aurs_04/")
df = pd.read_parquet(ins_path/"triangles/imgpaths-triangles-captions.parquet")
triangles = np.load(ins_path/"triangles/simplices.npy")
triangles_flat = rearrange(triangles, "n s p -> (n s) p")
captions = df.captions.values
paths = df.image_path.values

In [13]:
height = 512                       # default height of Stable Diffusion
width = 512                         # default width of Stable Diffusion
num_inference_steps = 20            # Number of denoising steps
guidance_scale = 7.5                # Scale for classifier-free guidance
generator = torch.manual_seed(42)   # Seed generator to create the inital latent noise
batch_size = 1

prompt_1 = captions[triangles[0,0,0]]
seed_1 = 42 
text_embed_1 = embed(prompt_1)
noise_1  = torch.randn(
  (batch_size, unet.in_channels, height // 8, width // 8),
  generator=torch.manual_seed(seed_1),
)

prompt_2 = captions[triangles[0,0,1]]
seed_2 = 1000
text_embed_2 = embed(prompt_2)
noise_2  = torch.randn(
  (batch_size, unet.in_channels, height // 8, width // 8),
  generator=torch.manual_seed(seed_2),
)

num_steps = 90
outdir = Path("output/test01/")
outdir.mkdir(exist_ok=True, parents=True)

for i, t in enumerate(np.linspace(0, 1, num_steps)):
    print("dreaming... ", i)

    
    # clean up unused memory
    gc.collect()
    torch.cuda.empty_cache()

    cond_embedding = slerp(float(t), text_embed_1, text_embed_2)
    init = slerp(float(t), noise_1, noise_2)
    image = diffuse(cond_embedding, init)
    outname = "frame%06d.jpg" % i
    outpath = (outdir/outname).as_posix()
    image.save(outpath)

dreaming...  0


0it [00:00, ?it/s]

dreaming...  1


0it [00:00, ?it/s]

dreaming...  2


...

0it [00:00, ?it/s]

dreaming...  88


0it [00:00, ?it/s]

dreaming...  89


0it [00:00, ?it/s]

In [15]:
! ffmpeg -r 10 -f image2 -s 1024x704 -i /content/cars/frame%06d.jpg -vcodec libx264 -crf 10 -pix_fmt yuv420p test01.mp4

ffmpeg: error while loading shared libraries: libopenh264.so.5: cannot open shared object file: No such file or directory


In [17]:
!ffmpeg -v

ffmpeg: error while loading shared libraries: libopenh264.so.5: cannot open shared object file: No such file or directory
